# Evaluacion de resultados

En este Notebook se muestran para la evaluacion de los resultados de una prediccion con un algoritmo de Machine Learning

## Dataset

### Descripción 
NSL-KDD is a data set suggested to solve some of the inherent problems of the KDD'99 data set which are mentioned in. Although, this new version of the KDD data set still suffers from some of the problems discussed by McHugh and may not be a perfect representative of existing real networks, because of the lack of public data sets for network-based IDSs, we believe it still can be applied as an effective benchmark data set to help researchers compare different intrusion detection methods. Furthermore, the number of records in the NSL-KDD train and test sets are reasonable. This advantage makes it affordable to run the experiments on the complete set without the need to randomly select a small portion. Consequently, evaluation results of different research work will be consistent and comparable.

## Ficheros de datos
* <span style="color:green">**KDDTrain+.ARFF**: The full NSL-KDD train set with binary labels in ARFF format</span>
* KDDTrain+.TXT: The full NSL-KDD train set including attack-type labels and difficulty level in CSV format
* KDDTrain+_20Percent.ARFF: A 20% subset of the KDDTrain+.arff file
* KDDTrain+_20Percent.TXT: A 20% subset of the KDDTrain+.txt file
* KDDTest+.ARFF: The full NSL-KDD test set with binary labels in ARFF format
* KDDTest+.TXT: The full NSL-KDD test set including attack-type labels and difficulty level in CSV format
* KDDTest-21.ARFF: A subset of the KDDTest+.arff file which does not include records with difficulty level of 21 out of 21
* KDDTest-21.TXT: A subset of the KDDTest+.txt file which does not include records with difficulty level of 21 out of 21

## Imports 

In [1]:
import arff
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

## Funciones Auxiliares

In [2]:
def load_kdd_dataset(data_path):
    """Lectura de DataSet NSL-KDD."""
    with open(data_path, 'r') as train_set:
        dataset = arff.load(train_set)
    attributes = [attr[0] for attr in dataset["attributes"]]
    return pd.DataFrame(dataset["data"], columns = attributes)

In [3]:
# Construcción de una función que relice le particionado completo
def train_val_test_split(df, rstate = 42, shuffle = True, stratify = None):
    strat = df[stratify] if stratify else None
    train_set, test_set = train_test_split(
        df, test_size = 0.4, random_state = rstate, shuffle = shuffle, stratify = strat)
    strat = test_set[stratify] if stratify else None
    val_set, test_set = train_test_split(
        test_set, test_size = 0.5, random_state = rstate, shuffle = shuffle, stratify = strat)
    return (train_set, val_set, test_set)

In [4]:
# construccion de un pipeline para los atributos numericos
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = "median")),
    ('rbst_scaler', RobustScaler())
])

In [5]:
# Transformador para codificar unicamente las columnas categoricas y devolver un DataFrame
class CustomOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self._oh = OneHotEncoder(sparse_output = False)
        self._columns = None
    def fit(self, x, y = None):
        x_cat = x.select_dtypes(include = ["object"])
        self._columns = pd.get_dummies(x_cat).columns
        self._oh.fit(x_cat)
        return self
    def transform(self, x, y = None):
        x_copy = x.copy()
        x_cat = x_copy.select_dtypes(include = ["object"])
        x_num = x_copy.select_dtypes(exclude = ["object"])
        x_cat_oh = self._oh.transform(x_cat)
        x_cat_oh = pd.DataFrame(x_cat_oh, columns = self._columns, index = x_copy.index)
        x_copy.drop(list(x_cat), axis = 1, inplace = True)
        return x_copy.join(x_cat_oh)

In [6]:
# Transformador queprepara todo el DataSet llamado Pipelines y transformadores personalizados
class DataFramePreparer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self._full_pipeline = None
        self._columns = None
    def fit(self, x, y = None):
        num_attribs = list(x.select_dtypes(exclude = ['object']))
        cat_attribs = list(x.select_dtypes(include = ['object']))
        self._full_pipeline = ColumnTransformer([
            ("num", num_pipeline, num_attribs),
            ("cat", CustomOneHotEncoder(), cat_attribs),
        ])
        self._full_pipeline.fit(x)
        self._columns = pd.get_dummies(x).columns
        return self
    def transform(self, x, y = None):
        x_copy = x.copy()
        x_prep = self._full_pipeline.transform(x_copy)
        return pd.DataFrame(x_prep, columns = self._columns, index = x_copy.index)

## Lectura del DataSet

In [7]:
df = load_kdd_dataset('datasets/datasets/NSL-KDD/KDDTrain+.arff')
df.head(10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0.0,tcp,ftp_data,SF,491.0,0.0,0,0.0,0.0,0.0,...,25.0,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0.0,udp,other,SF,146.0,0.0,0,0.0,0.0,0.0,...,1.0,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,26.0,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0.0,tcp,http,SF,232.0,8153.0,0,0.0,0.0,0.0,...,255.0,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0.0,tcp,http,SF,199.0,420.0,0,0.0,0.0,0.0,...,255.0,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
5,0.0,tcp,private,REJ,0.0,0.0,0,0.0,0.0,0.0,...,19.0,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,anomaly
6,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,9.0,0.04,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
7,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,15.0,0.06,0.07,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
8,0.0,tcp,remote_job,S0,0.0,0.0,0,0.0,0.0,0.0,...,23.0,0.09,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
9,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,13.0,0.05,0.06,0.00,0.00,1.00,1.00,0.00,0.00,anomaly


## Division del DataSet

In [8]:
# Division del DataSet en los diferentes subconjuntos
train_set, val_set, test_set = train_val_test_split(df)

In [9]:
print("Longitud del Training Set: ", len(train_set))
print("Longitud de la Validacion: ", len(val_set))
print("Longitud del Test Set: ", len(test_set))

Longitud del Training Set:  75583
Longitud de la Validacion:  25195
Longitud del Test Set:  25195


Para cada uno de los subconjunto, se separa las etiquetas pde las caracteristicas de entrada

In [11]:
# DataSet general 
x_df = df.drop("class", axis = 1)
y_df = df["class"].copy()

In [12]:
# DataSet de entrenamiento 
x_train = train_set.drop("class", axis = 1)
y_train = train_set["class"].copy()

In [13]:
# DataSet de validacion
x_val = val_set.drop("class", axis = 1)
y_val = val_set["class"].copy()

## Preparacion del DataSet

In [14]:
# instanciamos nuestro transformador personalizado
data_preparer = DataFramePreparer()

In [15]:
# Hacer el fit con el DataSet General para que adquiera tdos los valores posibles 
data_preparer.fit(x_df)

DataFramePreparer()

In [16]:
# Transformar el DataSet de entrenamiento
x_train_prep = data_preparer.transform(x_train)

In [17]:
x_train.head(10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
98320,0.0,icmp,ecr_i,SF,1032.0,0.0,0,0.0,0.0,0.0,...,210.0,65.0,0.31,0.01,0.31,0.00,0.0,0.0,0.0,0.0
8590,0.0,tcp,smtp,SF,1762.0,331.0,0,0.0,0.0,0.0,...,30.0,122.0,0.73,0.07,0.03,0.02,0.0,0.0,0.0,0.0
91385,0.0,icmp,eco_i,SF,8.0,0.0,0,0.0,0.0,0.0,...,2.0,126.0,1.00,0.00,1.00,0.25,0.0,0.0,0.0,0.0
54349,0.0,tcp,csnet_ns,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,18.0,0.07,0.07,0.00,0.00,1.0,1.0,0.0,0.0
69568,0.0,tcp,smtp,SF,1518.0,342.0,0,0.0,0.0,0.0,...,83.0,125.0,0.66,0.05,0.01,0.02,0.0,0.0,0.0,0.0
65413,0.0,tcp,http,SF,342.0,1148.0,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
106434,0.0,tcp,http,SF,247.0,11193.0,0,0.0,0.0,0.0,...,11.0,255.0,1.00,0.00,0.09,0.01,0.0,0.0,0.0,0.0
16874,0.0,tcp,http,SF,314.0,255.0,0,0.0,0.0,0.0,...,25.0,255.0,1.00,0.00,0.04,0.04,0.0,0.0,0.0,0.0
106132,0.0,udp,domain_u,SF,45.0,131.0,0,0.0,0.0,0.0,...,255.0,226.0,0.89,0.02,0.00,0.00,0.0,0.0,0.0,0.0
6183,0.0,tcp,uucp,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,15.0,0.06,0.07,0.00,0.00,1.0,1.0,0.0,0.0


In [18]:
x_train_prep.head(10)

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,flag_SF,flag_SH,land_0,land_1,logged_in_0,logged_in_1,is_host_login_0,is_host_login_1,is_guest_login_0,is_guest_login_1
98320,0.0,3.579710,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
8590,0.0,6.224638,0.641473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
91385,0.0,-0.130435,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
54349,0.0,-0.159420,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
69568,0.0,5.340580,0.662791,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
65413,0.0,1.079710,2.224806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
106434,0.0,0.735507,21.691860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
16874,0.0,0.978261,0.494186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
106132,0.0,0.003623,0.253876,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
6183,0.0,-0.159420,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [19]:
# Transformar el DataSet de validacion 
x_val_prep = data_preparer.transform(x_val)

In [20]:
x_val.head(10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
115297,0.0,tcp,uucp,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,15.0,0.06,0.05,0.00,0.00,1.0,1.0,0.0,0.0
7438,0.0,tcp,http,SF,318.0,3016.0,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
52597,5.0,tcp,smtp,SF,2274.0,334.0,0,0.0,0.0,0.0,...,104.0,52.0,0.50,0.05,0.01,0.00,0.0,0.0,0.0,0.0
10161,0.0,tcp,http,SF,292.0,1646.0,0,0.0,0.0,0.0,...,148.0,255.0,1.00,0.00,0.01,0.02,0.0,0.0,0.0,0.0
120044,0.0,tcp,finger,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,72.0,0.28,0.02,0.01,0.00,1.0,1.0,0.0,0.0
118005,0.0,tcp,imap4,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,7.0,0.03,0.06,0.00,0.00,1.0,1.0,0.0,0.0
80646,0.0,tcp,discard,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,11.0,0.04,0.07,0.00,0.00,1.0,1.0,0.0,0.0
110166,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,3.0,0.01,0.07,0.00,0.00,1.0,1.0,0.0,0.0
105046,0.0,tcp,http,SF,237.0,706.0,0,0.0,0.0,0.0,...,107.0,255.0,1.00,0.00,0.01,0.02,0.0,0.0,0.0,0.0
49742,0.0,udp,domain_u,SF,44.0,134.0,0,0.0,0.0,0.0,...,161.0,111.0,0.69,0.01,0.01,0.00,0.0,0.0,0.0,0.0


In [21]:
x_val_prep.head(10)

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,flag_SF,flag_SH,land_0,land_1,logged_in_0,logged_in_1,is_host_login_0,is_host_login_1,is_guest_login_0,is_guest_login_1
115297,0.0,-0.159420,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
7438,0.0,0.992754,5.844961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
52597,5.0,8.079710,0.647287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
10161,0.0,0.898551,3.189922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
120044,0.0,-0.159420,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
118005,0.0,-0.159420,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
80646,0.0,-0.159420,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
110166,0.0,-0.159420,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
105046,0.0,0.699275,1.368217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
49742,0.0,0.000000,0.259690,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
